In [1]:
import numpy as np
import pandas as pd
import datetime
#import random
#import os

In [2]:
# bigger dataset with q
path_to_sample_q = "/home/databox/data_sample_orig/bd_lab_large_sample_q.csv"

## smaller dataset from archive .gz
path_to_sample = "/home/databox/data_sample_orig/bd_lab_large_sample.csv"

In [3]:
# Column names -- from caption_file
headers = pd.read_csv("/home/databox/data_sample_orig/caption_file", sep = "\n", header = None,
                     squeeze = True)
headers = list(headers)
headers[:5]

['hash_number_A',
 'hash_tariff',
 'event_type',
 'event_sub',
 'network_service_direction']

In [4]:
## For loading random sample
#n = 92897157 #number of records in file
#s = 200000 #desired sample size
#skip = random.sample(range(n),n-s)

In [4]:
print("Loading started at {0}".format(datetime.datetime.now().time())) # start loadin

#
#nrows = 200000 # rows to load, add `nrows = nrows` into pd.read_csv

# for loading full datasert delete `nrows`
# full dataset
df = pd.read_csv(path_to_sample_q,  header = None, names = headers)

# 12 min
print("Loading finished at {0}".format(datetime.datetime.now().time())) # finish loading

Loading started at 06:41:33.475640


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,8,9,10,11,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Loading finished at 06:53:29.860600


In [5]:
len(df)

92897158

In [6]:
len(df["hash_number_A"].unique())

89265

In [7]:
len(df["hash_b_number"].unique())

2359746

In [9]:
df.head()

hash_number_A  hash_tariff   event_type    event_sub  \
0         148608       231925  network_ser  onnet_voice   
1         148608       231925  network_ser  onnet_voice   
2         148608       231925  network_ser  onnet_voice   
3         148608       231925  network_ser  onnet_voice   
4         148608       231925  network_ser  onnet_voice   

  network_service_direction     event_start_date hash_b_number  \
0                  Incoming   21JUL2017:14:35:00        367562   
1                  Outgoing   07JUN2017:11:46:44        367645   
2                  Outgoing   02AUG2017:11:24:13        367645   
3                  Incoming   10AUG2017:10:24:18        367155   
4                  Outgoing   10AUG2017:10:45:32        367645   

  number_B_category cost call_duration_minutes    ...      \
0                      0          0.6833333333    ...       
1                      0                     1    ...       
2                      0                     1    ...       
3                      0          0.2833333333    ...       
4                      0                     1    ...       

               interest_5 test_flag target facebook_user instagram_user  \
0  Business and economics         1      0             1              1   
1  Business and economics         1      0             1              1   
2  Business and economics         1      0             1              1   
3  Business and economics         1      0             1              1   
4  Business and economics         1      0             1              1   

  twitter_user youtube_user viber_user whatsapp_user skype_user  
0            1            1          1             1          1  
1            1            1          1             1          1  
2            1            1          1             1          1  
3            1            1          1             1          1  
4            1            1          1             1          1  

[5 rows x 30 columns]

#### Select and save Lviv only

In [12]:
# Delete rows without latitude -- where LAT = '.'

print(datetime.datetime.now().time())

# Write to new dataframe df_with_locat
df_with_locat = df[df['LAT'] != '.']

# Convert latitude and longitude from `str` into `float`
df_with_locat['LAT'] = pd.to_numeric(df_with_locat['LAT'], errors = 'coerce')
df_with_locat['LON'] = pd.to_numeric(df_with_locat['LON'], errors = 'coerce')

print(datetime.datetime.now().time())

07:03:24.584413


MemoryError: 

In [ ]:
# Lviv center coordinates
center_lat, center_lon = 49.84, 24.03  # Lviv
#center_lat, center_lon = 48.28, 25.94  # Chernivtsi


## calculate length in km of 1 degree for latitude and longitude
a = 6378.137; b = 6356.752 # semiaxis of Earth ellipsoid in km
e2 = (a**2 - b**2) / a**2 # eccentricity of Earth

# Convert degree to radians
def deg_to_rad(phi):
    return np.pi * phi / 180

# one degree of latitude and longitude in km
deg_lat_in_km = np.pi / 180 * a * (1-e2) /\
            (1 - e2*np.sin(deg_to_rad(center_lat)))**1.5
deg_lon_in_km = np.pi / 180 * a * np.cos(deg_to_rad(center_lat)) /\
             np.sqrt(1 - e2*np.sin(deg_to_rad(center_lat))**2 )

print("1° of latitude = {0:5.2f} km".format(deg_lat_in_km)) 
print("1° of longitude = {0:5.2f} km".format(deg_lon_in_km))

In [ ]:
#Convert distance in km to degrees
def km_to_lat(x):
    return x / deg_lat_in_km

def km_to_lon(x):
    return x / deg_lon_in_km

In [ ]:
# Radius of Lviv 
city_radius = 10 # km

# check if point is inside circle with radius `city_radius` and center in (center_lat, center_lon)
def isInCity_radius(lat, lon, center_lat, center_lon, city_radius):
    """True if (lat,lon) is in City, False otherwise"""
    return ((lat - center_lat)**2 / km_to_lat(city_radius)**2 +
            (lon - center_lon)**2 / km_to_lon(city_radius)**2) < 1


df_Lviv = df_with_locat[isInCity_radius(df_with_locat['LAT'],df_with_locat['LON'], 
                                 center_lat, center_lon, city_radius)]

In [ ]:
# export dataset with Lviv only to csv
df_Lviv.to_csv("large_sample_Lviv.csv")

##### Convert start date to DateTime type

In [9]:
df["event_start_date"][1]

' 07JUN2017:11:46:44'

In [10]:
# delete 1 blank space from begining of event_start entries
df["event_start_date"] = df["event_start_date"].apply(lambda x: x[1:])

In [ ]:
print("Loading started at {0}".format(datetime.datetime.now().time())) # start loading

# load with chunksize -- the same time (as simple load)
tp = pd.read_csv("/home/databox/data_sample_orig/bd_lab_large_sample_q.csv",
                iterator = True, chunksize = 100000)
df = pd.concat(tp, ignore_index=True)

print("Loading finished at {0}".format(datetime.datetime.now().time())) # finish loading
# 12 min to load full dataset


Loading started at 06:14:55.758796


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (6,8,9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.5/dist-packages/IPython/core/interact

In [28]:
df.head(3)
# where is column names?

hash_number_A  hash_tariff   event_type    event_sub  \
0         148608       231925  network_ser  onnet_voice   
1         148608       231925  network_ser  onnet_voice   
2         148608       231925  network_ser  onnet_voice   

  network_service_direction     event_start_date  hash_b_number  \
0                  Incoming   21JUL2017:14:35:00         367562   
1                  Outgoing   07JUN2017:11:46:44         367645   
2                  Outgoing   02AUG2017:11:24:13         367645   

  number_B_category  cost  call_duration_minutes    ...      \
0                       0               0.683333    ...       
1                       0               1.000000    ...       
2                       0               1.000000    ...       

               interest_5  test_flag  target  facebook_user instagram_user  \
0  Business and economics          1       0              1              1   
1  Business and economics          1       0              1              1   
2  Business and economics          1       0              1              1   

   twitter_user youtube_user viber_user whatsapp_user skype_user  
0             1            1          1             1          1  
1             1            1          1             1          1  
2             1            1          1             1          1  

[3 rows x 30 columns]

In [6]:
# number of records
len(df)

92897157

In [7]:
# number of abonents
len(df["148608"].unique())

89265

In [11]:
# dataset from hackathon
# 1 min 10 sec to load
df_small = pd.read_csv("/home/databox/userspace/bd_lab_small_sample.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,6,8,9,10,11,12,13,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
df_small.head()

hash_number_A  hash_tariff        event    event_sub  \
0      ﻿1001309        45807  network_ser  onnet_voice   
1        640090       221688  network_ser  onnet_voice   
2       1117021       223058  network_ser  onnet_voice   
3       2513535       187131  network_ser  onnet_voice   
4       1001309        45807  network_ser  onnet_voice   

  network_service_direction  event_start_date hash_b_number number_B_category  \
0                  Incoming  19JUN17:16:37:02        225968                     
1                  Outgoing  21AUG17:10:48:25        225899                     
2                  Incoming  11AUG17:09:55:56        225941                     
3                  Incoming  13AUG17:12:30:05        225867                     
4                  Incoming  19JUL17:08:31:25        225968                     

  cost call_duration_minutes  ...             LON device_type  \
0    0              0.416667  ...    24.707778541  smartphone   
1    0                     4  ...    35.122859331  smartphone   
2    0               1.63333  ...    25.755694444       phone   
3    0              0.266667  ...    23.855278159  smartphone   
4    0               11.3167  ...    24.707778541  smartphone   

  phone_price_category              interest_1      interest_2 interest_3  \
0                    2  Business and economics  News and Media    Hi-Tech   
1                    2                                                      
2                    1                                                      
3                    2                 Hi-Tech       Reference    Leisure   
4                    2  Business and economics  News and Media    Hi-Tech   

        interest_4       interest_5 test_flag target  
0        Reference  Home and family         0      0  
1                                           0      0  
2                                           0      0  
3  Home and family                          0      0  
4        Reference  Home and family         0      0  

[5 rows x 23 columns]

In [15]:
len(df_small)

10184607

In [16]:
df_small.columns

Index(['hash_number_A', 'hash_tariff', 'event', 'event_sub',
       'network_service_direction', 'event_start_date', 'hash_b_number',
       'number_B_category', 'cost', 'call_duration_minutes', 'data_volume_mb',
       'hash_accum_code', 'LAT', 'LON', 'device_type', 'phone_price_category',
       'interest_1', 'interest_2', 'interest_3', 'interest_4', 'interest_5',
       'test_flag', 'target'],
      dtype='object')

In [19]:
len(df_small["hash_number_A"].unique())

12901